In [1]:
%autosave 0

Autosave disabled


In [2]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [3]:
import os

host = os.getenv('TF_SERVING_HOST', 'localhost:8500')

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
url = 'https://raw.githubusercontent.com/ahmedokka29/ml-zoomcamp/main/11_capstone_1/data/Testing/glioma_tumor/image(1).jpg'

import numpy as np
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocessor(url):
    img = download_image(url)
    img = prepare_image(img,(224,224))

    x = np.array(img,dtype='float32')
    x = x/255
    X = np.array([x])
    return X

img = preprocessor(url)

In [5]:
url = 'https://raw.githubusercontent.com/ahmedokka29/ml-zoomcamp/main/11_capstone_1/data/Testing/glioma_tumor/image(1).jpg'
X = preprocessor(url)

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'brain-cancer-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['efficientnetv2-b3_input'].CopyFrom(np_to_protobuf(X))

In [9]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [10]:
preds = pb_response.outputs['dense_3'].float_val

In [11]:
classes = ['glioma_tumor',
           'meningioma_tumor',
           'no_tumor',
           'pituitary_tumor'
           ]

In [12]:
dict(zip(classes, preds))

{'glioma_tumor': 0.9992408752441406,
 'meningioma_tumor': 0.0003881289449054748,
 'no_tumor': 0.00037101731868460774,
 'pituitary_tumor': 4.493971861307955e-11}